<a href="https://colab.research.google.com/github/snoop2head/KoGPT-Joong-2/blob/main/wrangle_conditional_logit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Extract first and second largest emotion label and its logit values

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# DATA_PATH = "/content/drive/MyDrive/kogpt-joong-2-data/"
# DATA_PATH = "./gpt_data"
# PSEUDO_LABELED_FILE_NAME = "data_labeld_KoBERT_unhand.csv"
PSEUDO_LABELED_FILE_PATH = "/content/drive/MyDrive/kogpt-joong-2-data/data/data_final.csv"

In [ ]:
import os
pseudo_path = PSEUDO_LABELED_FILE_PATH
df_pseudo = pd.read_csv(pseudo_path, encoding="utf-8")
df_pseudo.drop(columns=["Unnamed: 0",], inplace=True)
df_pseudo.head()

,type,text,emotion,공포,놀람,분노,슬픔,중립,행복,혐오
0,대화,드디어 샀다.,행복,-1.976455,-0.497756,-1.301128,-1.327810,1.069673,5.177046,-1.353732
1,대화,뭐 하고 놀까.,행복,1.932847,-2.145293,-1.495267,2.116858,2.249198,-1.559769,-0.994669
2,대화,오늘은 지각 안했어!.,행복,-1.364891,-0.819980,-1.040110,-2.304478,1.620021,5.133489,-1.207578
3,대화,오늘 점심 제육볶음 이래!.,행복,-1.533068,-0.162358,-0.926851,-2.654852,2.069342,4.444376,-1.225262
4,대화,나 먼저 갈게.,행복,-0.676803,-1.909779,-2.075403,0.887898,1.514351,4.160100,-2.037049


In [ ]:
df_pseudo.tail()

,type,text,emotion,공포,놀람,분노,슬픔,중립,행복,혐오
494091,노래 가사,눈물은 뚝 눈물은 뚝 얼싸 안고 춤을 춰보자 춘향아 춘향아 눈물을 거둬라,중립,-1.791054,-2.507724,-0.895269,0.827443,3.551021,1.050611,-0.640308
494092,노래 가사,어차피 다 지난 일인걸 더 이상 무슨 말이 필요해 서둘지 말고 한걸음씩 즐겨봐,행복,-2.078648,-1.968988,-2.018869,0.097698,3.121234,3.633747,-1.375754
494093,노래 가사,참 미안해 이제 그만 아프지 마 제발 힘들었지 떠나지마 참 미안해 이러지마,슬픔,-0.457692,-0.860459,-0.770534,5.504443,-1.656590,-0.837802,-1.208101
494094,노래 가사,내 못난 마음 꿈에서는 다 용서해 주세요,슬픔,-0.706598,-2.820624,-1.454884,4.223856,0.895535,1.043546,-1.225848
494095,노래 가사,이 날이 오기 전에 난 어떻게 품는 줄 몰랐죠,슬픔,1.442261,1.019177,-2.571514,3.298930,0.045752,0.175383,-3.429953


In [ ]:
emotion_columns = list(df_pseudo.columns[3:])
emotion_columns

['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오']

In [ ]:
df_pseudo.iloc[0][emotion_columns]

공포    -1.97646
놀람   -0.497756
분노    -1.30113
슬픔    -1.32781
중립     1.06967
행복     5.17705
혐오    -1.35373
Name: 0, dtype: object

In [ ]:
df_pseudo[emotion_columns].describe()

,공포,놀람,분노,슬픔,중립,행복,혐오
count,494096.000000,494096.000000,494096.000000,494096.000000,494096.000000,494096.000000,494096.000000
mean,-0.156072,-0.234796,-0.673835,0.425831,1.146357,0.284349,-0.805222
std,1.554521,1.572275,1.625884,2.204516,2.093880,2.968663,1.422963
min,-3.940771,-4.007143,-3.883015,-3.873175,-4.176189,-5.246027,-4.317473
25%,-1.291746,-1.356023,-1.824280,-1.335475,-0.335367,-2.066455,-1.736851
50%,-0.521597,-0.566369,-1.162270,-0.233296,1.130529,-0.674364,-1.117808
75%,0.650117,0.612193,0.081993,1.901538,2.892123,2.431469,-0.183897
max,5.573719,5.327038,5.425459,5.750891,5.254232,6.463407,4.181581


In [ ]:
df_pseudo_argmaxed = df_pseudo.copy()
# drop emotion column
df_pseudo_argmaxed.drop(columns=["emotion"], inplace=True)
df_pseudo_argmaxed["first_emotion"] = df_pseudo[emotion_columns].idxmax(axis=1)
df_pseudo_argmaxed.head()

,type,text,공포,놀람,분노,슬픔,중립,행복,혐오,first_emotion
0,대화,드디어 샀다.,-1.976455,-0.497756,-1.301128,-1.327810,1.069673,5.177046,-1.353732,행복
1,대화,뭐 하고 놀까.,1.932847,-2.145293,-1.495267,2.116858,2.249198,-1.559769,-0.994669,중립
2,대화,오늘은 지각 안했어!.,-1.364891,-0.819980,-1.040110,-2.304478,1.620021,5.133489,-1.207578,행복
3,대화,오늘 점심 제육볶음 이래!.,-1.533068,-0.162358,-0.926851,-2.654852,2.069342,4.444376,-1.225262,행복
4,대화,나 먼저 갈게.,-0.676803,-1.909779,-2.075403,0.887898,1.514351,4.160100,-2.037049,행복


In [ ]:
# show row with nan value
df_pseudo_argmaxed[df_pseudo_argmaxed["first_emotion"].isna()]

,type,text,공포,놀람,분노,슬픔,중립,행복,혐오,first_emotion


In [ ]:
df_pseudo_argmaxed.head()

,type,text,공포,놀람,분노,슬픔,중립,행복,혐오,first_emotion
0,대화,드디어 샀다.,-1.976455,-0.497756,-1.301128,-1.327810,1.069673,5.177046,-1.353732,행복
1,대화,뭐 하고 놀까.,1.932847,-2.145293,-1.495267,2.116858,2.249198,-1.559769,-0.994669,중립
2,대화,오늘은 지각 안했어!.,-1.364891,-0.819980,-1.040110,-2.304478,1.620021,5.133489,-1.207578,행복
3,대화,오늘 점심 제육볶음 이래!.,-1.533068,-0.162358,-0.926851,-2.654852,2.069342,4.444376,-1.225262,행복
4,대화,나 먼저 갈게.,-0.676803,-1.909779,-2.075403,0.887898,1.514351,4.160100,-2.037049,행복


In [ ]:
df_pseudo_argmaxed_logit = df_pseudo_argmaxed.copy()

# drop nan values
df_pseudo_argmaxed_logit.dropna(inplace=True)

first_logit = df_pseudo_argmaxed_logit[emotion_columns].values.max(1)
second_logit = df_pseudo_argmaxed_logit[emotion_columns].apply(lambda row: row.nlargest(2).values[-1],axis=1) # https://stackoverflow.com/a/48638056/8380469

df_pseudo_argmaxed_logit["first_logit"] = first_logit
df_pseudo_argmaxed_logit["second_logit"] = second_logit

second_emotion = df_pseudo_argmaxed_logit.apply(lambda row: row.iloc[:-2][row.iloc[:-2] == row["second_logit"]].index[0], axis=1)
df_pseudo_argmaxed_logit["second_emotion"] = second_emotion
df_pseudo_argmaxed_logit.head()

,type,text,공포,놀람,분노,슬픔,중립,행복,혐오,first_emotion,first_logit,second_logit,second_emotion
0,대화,드디어 샀다.,-1.976455,-0.497756,-1.301128,-1.327810,1.069673,5.177046,-1.353732,행복,5.177046,1.069673,중립
1,대화,뭐 하고 놀까.,1.932847,-2.145293,-1.495267,2.116858,2.249198,-1.559769,-0.994669,중립,2.249198,2.116858,슬픔
2,대화,오늘은 지각 안했어!.,-1.364891,-0.819980,-1.040110,-2.304478,1.620021,5.133489,-1.207578,행복,5.133489,1.620021,중립
3,대화,오늘 점심 제육볶음 이래!.,-1.533068,-0.162358,-0.926851,-2.654852,2.069342,4.444376,-1.225262,행복,4.444376,2.069342,중립
4,대화,나 먼저 갈게.,-0.676803,-1.909779,-2.075403,0.887898,1.514351,4.160100,-2.037049,행복,4.160100,1.514351,중립


In [ ]:
display(df_pseudo_argmaxed_logit.head())
df_pseudo_argmaxed_logit.shape

,type,text,공포,놀람,분노,슬픔,중립,행복,혐오,first_emotion,first_logit,second_logit,second_emotion
0,대화,드디어 샀다.,-1.976455,-0.497756,-1.301128,-1.327810,1.069673,5.177046,-1.353732,행복,5.177046,1.069673,중립
1,대화,뭐 하고 놀까.,1.932847,-2.145293,-1.495267,2.116858,2.249198,-1.559769,-0.994669,중립,2.249198,2.116858,슬픔
2,대화,오늘은 지각 안했어!.,-1.364891,-0.819980,-1.040110,-2.304478,1.620021,5.133489,-1.207578,행복,5.133489,1.620021,중립
3,대화,오늘 점심 제육볶음 이래!.,-1.533068,-0.162358,-0.926851,-2.654852,2.069342,4.444376,-1.225262,행복,4.444376,2.069342,중립
4,대화,나 먼저 갈게.,-0.676803,-1.909779,-2.075403,0.887898,1.514351,4.160100,-2.037049,행복,4.160100,1.514351,중립


(494096, 13)

In [ ]:
# show distribution of each emotion
display(df_pseudo_argmaxed_logit["first_emotion"].value_counts())
display(df_pseudo_argmaxed_logit["second_emotion"].value_counts())

중립    154891
행복    112030
슬픔     98877
놀람     40088
공포     33551
분노     31297
혐오     23362
Name: first_emotion, dtype: int64

중립    127928
공포     76228
행복     72540
슬픔     66883
놀람     63391
혐오     46243
분노     40883
Name: second_emotion, dtype: int64

In [ ]:
df_pseudo_argmaxed_logit[["first_logit", "second_logit"]].describe()

,first_logit,second_logit
count,494096.000000,494096.000000
mean,3.838390,1.425989
std,1.163832,0.898886
min,0.451860,-0.922410
25%,2.979933,0.781728
50%,3.764594,1.449031
75%,4.610926,2.070051
max,6.463407,3.871762


In [ ]:
df_pseudo_argmaxed_logit.to_csv("/content/drive/MyDrive/kogpt-joong-2-data/data/data_final_argmaxed.csv", index=False, encoding="utf-8")

In [ ]:
df_pseudo_argmaxed_logit.sample(10)

,type,text,공포,놀람,분노,슬픔,중립,행복,혐오,first_emotion,first_logit,second_logit,second_emotion
418627,감성글,볕은 들뜬 듯 요란하고 주위가 원래 소란했던가.,0.440130,2.321883,-1.255807,-2.061872,2.006249,0.882345,-1.961903,놀람,2.321883,2.006249,중립
6482,대학생 커뮤니티 댓글,마음까지도 간지럽히는 음식 뭐있을까? 나만 이런가?,-0.210554,-0.748452,-1.573301,0.912662,3.943931,-2.155027,-0.686667,중립,3.943931,0.912662,슬픔
492822,노래 가사,너무 모르고 있죠 이미 떠난 사인데 언젠가 올 거란 생각에 마냥 웃고만 있죠,0.629739,-1.389954,-1.588779,5.134431,-0.632014,0.165006,-1.962485,슬픔,5.134431,0.629739,공포
390664,수필,장마나 태풍이 휩쓸고 가면 서민들의 밥상을 책임지는 채소 값은 널을 뛰기 마련이다,1.085873,-0.867665,0.269248,0.317287,2.535652,-2.730529,-0.471004,중립,2.535652,1.085873,공포
345498,수필,"지난 한 해 동안, 여러 명의 지인들을 떠나보냈다",-0.202123,-0.342046,-1.571968,5.315048,-0.872584,-0.380076,-2.007023,슬픔,5.315048,-0.202123,공포
383948,수필,"죽은 애 혼자 두믄 어때요? 하고, 다시 톡 쏘는 간호부의 목소리는 어름같이 싸늘하였다",1.941537,-1.005561,1.503038,1.268815,0.486743,-3.344244,-0.577468,공포,1.941537,1.503038,분노
387513,수필,대신 집은 아파트가 별로 없고 서양식 2층 주택이 많더군요,-0.431156,2.306866,-2.343176,0.989935,2.104830,-1.364821,-1.421207,놀람,2.306866,2.104830,중립
232488,수필,자다가 눈을 뜨면 한번 더 쳐다보고 싶어서였다,0.487719,0.448604,2.757120,-0.885281,0.356124,-2.732742,-0.416797,분노,2.757120,0.487719,공포
401682,대학생 커뮤니티 댓글,"그대 내게로 오는길 그대의 향기에 취하여, 그대의 숨결에 따라 춤을 추리라",-1.628008,-1.489305,-1.727855,-1.490288,2.755331,4.782099,-1.499402,행복,4.782099,2.755331,중립
89766,채팅,항상 매주마다 유느님 모습들이 재밌게 보여주시니감사합니다,-1.408628,-1.295597,-1.311791,-1.474911,-0.077872,6.371384,-0.875117,행복,6.371384,-0.077872,중립


In [ ]:
df_pseudo_argmaxed_logit[df_pseudo_argmaxed_logit["type"]=="대화"].shape

### Use 감성대화말뭉치 and assign arbitrary logit value

In [ ]:
pd.set_option("display.max_colwidth", 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# df_pseudo.sample(100)

In [ ]:
감성대화말뭉치 = "/content/drive/MyDrive/kogpt-joong-2-data/라벨링 검수 완료/감성대화 말뭉치.csv"
# 감성대화말뭉치 = "./gpt_data/감성대화 말뭉치.csv"
감성대화말뭉치_df = pd.read_csv(감성대화말뭉치, encoding="utf-8")
감성대화말뭉치_df.drop(columns=["Unnamed: 0",], inplace=True)
감성대화말뭉치_df.head()

,번호,text,emotion
0,44164,아내가 드디어 출산하게 되어서 정말 신이 나. 아 지금 정말 신이 나. 아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,행복
1,3926,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야. 건강할 때 관리 좀 잘할걸 하는 생각이 들더라고. 약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,공포
2,50882,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워. 아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼. 중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야. 선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,놀람
3,31303,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해. 퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 수 있어서 너무 행복해. 회사생활을 열심히 해서 계속 월급을 받을거야!,행복
4,47200,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어. 빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,행복


In [ ]:
감성대화말뭉치_df.shape

(46009, 3)

In [ ]:
감성대화말뭉치_df_copy = 감성대화말뭉치_df.copy()
감성대화말뭉치_df_copy["type"] = "대화"

# drop 번호
감성대화말뭉치_df_copy.drop(columns=["번호",], inplace=True)
감성대화말뭉치_df_copy.head()

# change column name from emotion to first_emotion
감성대화말뭉치_df_copy.rename(columns={"emotion": "first_emotion"}, inplace=True)
감성대화말뭉치_df_copy.head()

,text,first_emotion,type
0,아내가 드디어 출산하게 되어서 정말 신이 나. 아 지금 정말 신이 나. 아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,행복,대화
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야. 건강할 때 관리 좀 잘할걸 하는 생각이 들더라고. 약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,공포,대화
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워. 아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼. 중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야. 선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,놀람,대화
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해. 퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 수 있어서 너무 행복해. 회사생활을 열심히 해서 계속 월급을 받을거야!,행복,대화
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어. 빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,행복,대화


In [ ]:
감성대화말뭉치_df_copy["first_emotion"].value_counts()

슬픔    15216
공포     8236
분노     7793
행복     7572
놀람     7192
Name: first_emotion, dtype: int64

In [ ]:
df_pseudo_argmaxed_logit[["first_logit"]].std()

first_logit    1.015295
dtype: float64

In [ ]:
first_logit_random = np.random.normal(
    loc=df_pseudo_argmaxed_logit[["first_logit"]].mean(),
    scale=df_pseudo_argmaxed_logit[["first_logit"]].std(),
    size=len(감성대화말뭉치_df_copy)
    )
first_logit_random[:10]

array([3.16198012, 3.6157767 , 3.95329929, 2.37625461, 2.12877156,
       4.41939916, 2.74179986, 2.52215667, 2.45359625, 2.7370469 ])

In [ ]:
감성대화말뭉치_df_copy["first_logit"] = first_logit_random
감성대화말뭉치_df_copy.head()

,text,first_emotion,type,first_logit
0,아내가 드디어 출산하게 되어서 정말 신이 나. 아 지금 정말 신이 나. 아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,행복,대화,3.161980
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야. 건강할 때 관리 좀 잘할걸 하는 생각이 들더라고. 약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,공포,대화,3.615777
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워. 아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼. 중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야. 선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,놀람,대화,3.953299
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해. 퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 수 있어서 너무 행복해. 회사생활을 열심히 해서 계속 월급을 받을거야!,행복,대화,2.376255
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어. 빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,행복,대화,2.128772


In [ ]:
# concat df_pseudo_argmaxed_logit and 감성대화말뭉치_df_copy
df_pseudo_argmaxed_logit_and_감성대화말뭉치_df_copy = pd.concat([df_pseudo_argmaxed_logit, 감성대화말뭉치_df_copy], axis=0)
df_pseudo_argmaxed_logit_and_감성대화말뭉치_df_copy.drop(columns=emotion_columns, inplace=True)
df_pseudo_argmaxed_logit_and_감성대화말뭉치_df_copy.head()

,type,text,first_emotion,first_logit,second_logit,second_emotion
0,수필,자신을 온전히 선물로 바칠 가족이 있다는 것은 얼마나 복된 일인가,중립,1.893878,1.586071,놀람
1,수필,너희 어머닌 생전에도 방앗간 떡은 안 먹었다,혐오,1.880819,1.766544,중립
2,수필,그때 자투리 공원에서 신선하게 미소 짓는 배롱나무꽃과 눈이 맞았다,행복,3.965406,1.533189,중립
3,수필,그래서 관심조차 없어 무심히 지나쳤을 뿐이다,슬픔,3.875700,1.038357,혐오
4,수필,태양도 달도 별도 지구도 하늘도 바다도 다 돈다,중립,2.371011,0.970357,놀람


In [ ]:
df_pseudo_argmaxed_logit_and_감성대화말뭉치_df_copy.tail()

,type,text,first_emotion,first_logit,second_logit,second_emotion
46004,대화,부동산 임대 소득으로 현재 여유롭게 살 수 있어서 좋단다. 그래서 마음의 안정을 가질 수 있었던 것 같아. 돈을 함부로 쓰지 말고 지금 현재에 만족하면서 지내야지.,행복,4.866405,NaN,NaN
46005,대화,폐결핵은 이미 완치된 것 같은데 약을 한 달이나 더 먹으라고 하네? 아직 안 나은 게 아닌지 걱정돼. 이대로 혹시 더 심해져서 죽게 되는 건 아닐까? 의사 선생님에게 악화될 확률은 얼마나 되는지 정확히 물어봐야겠어.,공포,2.725927,NaN,NaN
46006,대화,연애하고 싶은데 소개팅만 나가면 꽝이야. 이러다가 난 결혼 못 하고 늙어 죽을 거야. 내가 게을러서 다른 방법을 아직 생각하진 못했어. 댄스 동호회는 좀 괜찮은 것 같은데 이참에 가입해볼까?,분노,3.358419,NaN,NaN
46007,대화,은행 대출이 막혀서 생활비를 구할 수가 없어. 이제 어떻게 살아야 하나 막막해. 당장 내일 밥은 어떡하고 내 병원비는 어쩌지. 어떻게 해야 할 지 모르겠어. 눈 앞이 캄캄해.,분노,2.967055,NaN,NaN
46008,대화,자식들은 키워 준 은혜도 모르고 내게 오지도 않네. 너무 외롭고 슬퍼. 응 이제 다들 독립해가지고 명절 아니면 찾아오질 않아. 안부전화라도 해주면 좋을 텐데. 이런 감정이 들 때면 옛날 생각이 많이 나. 옛날에는 그게 당연한 것인 줄 알았지.,분노,3.741671,NaN,NaN


In [ ]:
# reset index of df_pseudo_argmaxed_logit_and_감성대화말뭉치_df_copy
df_pseudo_argmaxed_logit_and_감성대화말뭉치_df_copy.reset_index(drop=True, inplace=True)
df_pseudo_argmaxed_logit_and_감성대화말뭉치_df_copy.tail()

,type,text,first_emotion,first_logit,second_logit,second_emotion
441973,대화,부동산 임대 소득으로 현재 여유롭게 살 수 있어서 좋단다. 그래서 마음의 안정을 가질 수 있었던 것 같아. 돈을 함부로 쓰지 말고 지금 현재에 만족하면서 지내야지.,행복,4.866405,NaN,NaN
441974,대화,폐결핵은 이미 완치된 것 같은데 약을 한 달이나 더 먹으라고 하네? 아직 안 나은 게 아닌지 걱정돼. 이대로 혹시 더 심해져서 죽게 되는 건 아닐까? 의사 선생님에게 악화될 확률은 얼마나 되는지 정확히 물어봐야겠어.,공포,2.725927,NaN,NaN
441975,대화,연애하고 싶은데 소개팅만 나가면 꽝이야. 이러다가 난 결혼 못 하고 늙어 죽을 거야. 내가 게을러서 다른 방법을 아직 생각하진 못했어. 댄스 동호회는 좀 괜찮은 것 같은데 이참에 가입해볼까?,분노,3.358419,NaN,NaN
441976,대화,은행 대출이 막혀서 생활비를 구할 수가 없어. 이제 어떻게 살아야 하나 막막해. 당장 내일 밥은 어떡하고 내 병원비는 어쩌지. 어떻게 해야 할 지 모르겠어. 눈 앞이 캄캄해.,분노,2.967055,NaN,NaN
441977,대화,자식들은 키워 준 은혜도 모르고 내게 오지도 않네. 너무 외롭고 슬퍼. 응 이제 다들 독립해가지고 명절 아니면 찾아오질 않아. 안부전화라도 해주면 좋을 텐데. 이런 감정이 들 때면 옛날 생각이 많이 나. 옛날에는 그게 당연한 것인 줄 알았지.,분노,3.741671,NaN,NaN


In [ ]:
df_pseudo_argmaxed_logit_and_감성대화말뭉치_df_copy.to_csv("./gpt_data/data_labeld_KoBERT_unhand_argmaxed_and_감성대화말뭉치.csv", index=False, encoding="utf-8")